In [176]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import shutil
from PIL import Image
import glob
import shutil

# split cases on distribution

In [177]:
all_data = pd.read_csv('/A_LUAD/project/final_project/data/case_info.csv')
grouped = all_data.groupby('ajcc_pathologic_stage')
## get test cases
train_val_temp = grouped.apply(lambda x: x.sample(frac=0.8))
train_val_id = train_val_temp['case_submitter_id'].tolist()
train_val = pd.DataFrame(train_val_id, columns=['case_submitter_id'])
train_val = train_val.merge(all_data, how='inner', on='case_submitter_id')
test_id = set(all_data['case_submitter_id']).difference(train_val['case_submitter_id'])
test = pd.DataFrame(test_id, columns=['case_submitter_id'])
test = test.merge(all_data, how='inner', on='case_submitter_id')

In [178]:
## get validation cases
val_temp = train_val_temp.apply(lambda x: x.sample(frac=0.2))
val_id = val_temp['case_submitter_id'].tolist()
val = pd.DataFrame(val_id, columns=['case_submitter_id'])
val = val.merge(all_data, how='inner', on='case_submitter_id')
# print(val['image_num'].sum(),'\t',rest['image_num'].sum(),'\t',val['image_num'].sum()/rest['image_num'].sum())

In [179]:
## get training cases
train_id = set(train_val_temp['case_submitter_id']).difference(val['case_submitter_id'])
train = pd.DataFrame(train_id, columns=['case_submitter_id'])
train = train.merge(all_data, how='inner', on='case_submitter_id')
print(train['image_num'].sum())

3075


# unnest data

## get test imgae csv

In [180]:
nested_data = test

In [181]:
image_index = []
for i in nested_data['image_num']:
  temp = list(range(1,i+1))
  image_index.append(temp)
print(image_index)

[[1, 2], [1, 2, 3, 4, 5, 6], [1, 2], [1, 2], [1], [1], [1, 2, 3], [1], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13], [1, 2], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34], [1], [1], [1], [1], [1, 2, 3, 4, 5, 6, 7], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42], [1, 

In [182]:
nested_data['image_index'] = image_index

In [183]:
unnested_data = nested_data.explode('image_index')

In [184]:
unnested_data['image_id'] = unnested_data['case_submitter_id'] + '_' + unnested_data['image_index'].astype(str)

In [185]:
cleaned_unnested_data=unnested_data.drop(columns=['image_num','image_index'])
cleaned_unnested_data=cleaned_unnested_data[['image_id','case_submitter_id','vital_status','survival_days','survival_year','group','ajcc_pathologic_stage','ajcc_staging_system_edition','ajcc_pathologic_t','ajcc_pathologic_m','ajcc_pathologic_n','primary_diagnosis']]

In [186]:
cleaned_unnested_data.to_csv('/A_LUAD/project/final_project/data/temp/test_images.csv', index=False)

## get val imgae csv

In [187]:
nested_data = val

In [188]:
image_index = []
for i in nested_data['image_num']:
  temp = list(range(1,i+1))
  image_index.append(temp)
print(image_index)

[[1, 2, 3, 4, 5, 6, 7, 8], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39], [1, 2, 3, 4, 5, 6, 7, 8, 9], [1, 2, 3, 4, 5, 

In [189]:
nested_data['image_index'] = image_index

In [190]:
unnested_data = nested_data.explode('image_index')

In [191]:
unnested_data['image_id'] = unnested_data['case_submitter_id'] + '_' + unnested_data['image_index'].astype(str)

In [192]:
cleaned_unnested_data=unnested_data.drop(columns=['image_num','image_index'])
cleaned_unnested_data=cleaned_unnested_data[['image_id','case_submitter_id','vital_status','survival_days','survival_year','group','ajcc_pathologic_stage','ajcc_staging_system_edition','ajcc_pathologic_t','ajcc_pathologic_m','ajcc_pathologic_n','primary_diagnosis']]

In [193]:
cleaned_unnested_data.to_csv('/A_LUAD/project/final_project/data/temp/val_images.csv', index=False)

## get train imgae csv

In [194]:
nested_data = train

In [195]:
image_index = []
for i in nested_data['image_num']:
  temp = list(range(1,i+1))
  image_index.append(temp)
print(image_index)

[[1], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25], [1, 2, 3, 4, 5], [1], [1], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133], [1, 2], [1, 2, 3, 4, 5], [1, 2], [1, 2, 3, 4, 5, 6, 7, 8], [1, 2, 3, 4, 5, 6, 7, 8], [1], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45], [1], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14,

In [196]:
nested_data['image_index'] = image_index

In [197]:
unnested_data = nested_data.explode('image_index')

In [198]:
unnested_data['image_id'] = unnested_data['case_submitter_id'] + '_' + unnested_data['image_index'].astype(str)

In [199]:
cleaned_unnested_data=unnested_data.drop(columns=['image_num','image_index'])
cleaned_unnested_data=cleaned_unnested_data[['image_id','case_submitter_id','vital_status','survival_days','survival_year','group','ajcc_pathologic_stage','ajcc_staging_system_edition','ajcc_pathologic_t','ajcc_pathologic_m','ajcc_pathologic_n','primary_diagnosis']]

In [200]:
cleaned_unnested_data.to_csv('/A_LUAD/project/final_project/data/temp/train_images.csv', index=False)

# split images to directory

In [201]:
# read csv files, get all information
image_paths = glob.glob("D:/A_LUAD/project/final_project/data/raw_data/*.png")
print("Total Observations:\t", len(image_paths))

Total Observations:	 4726


## get train dir

In [202]:
data = pd.read_csv('/A_LUAD/project/final_project/data/temp/train_images.csv')
all_images_path = ['D:/A_LUAD/project/final_project/data/raw_data/{}.png'.format(i) for i in data['image_id'].values]

In [203]:
gb = data.groupby('group')
# group1 = grouped.getgroup('1')
group1 = gb[['image_id','group']].get_group(1)['image_id'].tolist()
group2 = gb[['image_id','group']].get_group(2)['image_id'].tolist()
group3 = gb[['image_id','group']].get_group(3)['image_id'].tolist()
groups = []
groups.append(group1)
groups.append(group2)
groups.append(group3)
print(len(groups),len(groups[0]),len(groups[1]),len(groups[2]))

3 216 2098 761


In [204]:
new_root_path = '/A_LUAD/project/final_project/data/temp/train'
old_root_path = 'D:/A_LUAD/project/data/processed_images'

for i, group in enumerate(groups):
  group_name = 'group'+str(i+1)
  group_path = os.path.join(new_root_path,group_name)
  print(group_name)
  n = 0
  if not os.path.exists(group_path):
    os.makedirs(group_path)
  for image_id in group:
    image_name = image_id+'.png'
    image_old_path=os.path.join(old_root_path,image_name)
    image_new_path=os.path.join(group_path,image_name)
    shutil.copy(image_old_path,image_new_path)


group1
group2
group3


## get test dir

In [205]:
data = pd.read_csv('/A_LUAD/project/final_project/data/temp/test_images.csv')
all_images_path = ['D:/A_LUAD/project/final_project/data/raw_data/{}.png'.format(i) for i in data['image_id'].values]

In [206]:
gb = data.groupby('group')
# group1 = grouped.getgroup('1')
group1 = gb[['image_id','group']].get_group(1)['image_id'].tolist()
group2 = gb[['image_id','group']].get_group(2)['image_id'].tolist()
group3 = gb[['image_id','group']].get_group(3)['image_id'].tolist()
groups = []
groups.append(group1)
groups.append(group2)
groups.append(group3)
print(len(groups),len(groups[0]),len(groups[1]),len(groups[2]))

3 95 665 239


In [207]:
new_root_path = '/A_LUAD/project/final_project/data/temp/test'
old_root_path = 'D:/A_LUAD/project/data/processed_images'

for i, group in enumerate(groups):
  group_name = 'group'+str(i+1)
  group_path = os.path.join(new_root_path,group_name)
  print(group_name)
  n = 0
  if not os.path.exists(group_path):
    os.makedirs(group_path)
  for image_id in group:
    image_name = image_id+'.png'
    image_old_path=os.path.join(old_root_path,image_name)
    image_new_path=os.path.join(group_path,image_name)
    shutil.copy(image_old_path,image_new_path)


group1
group2
group3


## get val dir

In [208]:
data = pd.read_csv('/A_LUAD/project/final_project/data/temp/val_images.csv')
all_images_path = ['D:/A_LUAD/project/final_project/data/raw_data/{}.png'.format(i) for i in data['image_id'].values]

In [209]:
gb = data.groupby('group')
# group1 = grouped.getgroup('1')
group1 = gb[['image_id','group']].get_group(1)['image_id'].tolist()
group2 = gb[['image_id','group']].get_group(2)['image_id'].tolist()
group3 = gb[['image_id','group']].get_group(3)['image_id'].tolist()
groups = []
groups.append(group1)
groups.append(group2)
groups.append(group3)
print(len(groups),len(groups[0]),len(groups[1]),len(groups[2]))

3 281 306 65


In [210]:
new_root_path = '/A_LUAD/project/final_project/data/temp/val'
old_root_path = 'D:/A_LUAD/project/data/processed_images'

for i, group in enumerate(groups):
  group_name = 'group'+str(i+1)
  group_path = os.path.join(new_root_path,group_name)
  print(group_name)
  n = 0
  if not os.path.exists(group_path):
    os.makedirs(group_path)
  for image_id in group:
    image_name = image_id+'.png'
    image_old_path=os.path.join(old_root_path,image_name)
    image_new_path=os.path.join(group_path,image_name)
    shutil.copy(image_old_path,image_new_path)


group1
group2
group3
